In [1]:
from googleapiclient.discovery import build
import pandas as pd
import re
import isodate

In [2]:
def extract(api_key, playlist_id):
    youtube = build('youtube', 'v3', developerKey=api_key)
    videos = []
    next_page_token = None

    while True:
        # YouTube API
        request = youtube.playlistItems().list(
            part = 'snippet',
            playlistId = playlist_id,
            maxResults = 50,  # YouTube API max limit
            pageToken = next_page_token)
        response = request.execute()

        # extract data
        for item in response['items']:
            snippet = item['snippet']
            video_id = snippet['resourceId']['videoId']
            title = snippet['title']
            description = snippet['description']
            channel_title = snippet['channelTitle']
            video_url = f"https://www.youtube.com/watch?v={video_id}"       

            videos.append({
                'Title': title,
                'Description': description,
                'Channel': channel_title,
                'URL': video_url
            })

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    return pd.DataFrame(videos)

In [3]:
api_key = "Your_api_key"
playlist_ids = ["PL4_bo90i-4GItp18iodTK2cv-Mzn0k1es", "PLRCzrSHS5u_HQuXr15gOMCIK-E-57A5vd","PL0ePmFx_GS-I6LJcY13e1UNdD1TJgaJvU",
                "PLAhq2Pnx-VYK6Ou-M6Fm2I3FJ2j_b3Oww", "PLAhq2Pnx-VYIZPcA-m7OieTUWcQ01vVPY", "PLAhq2Pnx-VYKbgFAxRxaNVG2p0IOsukgX",
                "PLAhq2Pnx-VYIFyJQnvl3gTEqlvRnWwnfE", "PLAhq2Pnx-VYIr4CjRor62E2UJMlGILErF", "PLAhq2Pnx-VYIZCgft9hSAHf_OuoEfsb3H",
                "PLAhq2Pnx-VYIdb7Z-Uw0IZTRCZMS3Ytrr", "PLAhq2Pnx-VYJSf0odwGierUaLSIaNvUUD",
                "PLAhq2Pnx-VYIMlJKtcSrYO3TXKu8KvYxv", "PLAhq2Pnx-VYLeu7yjrNn4KKWz8H15zFRd",
               ]
data = pd.DataFrame()

In [4]:
for ID in playlist_ids:
    df = extract(api_key,ID)
    data = pd.concat([data, df], ignore_index=True)

In [5]:
data.head(5)

,Title,Description,Channel,URL
0,بحبك يا .... | الدحيح,تدور الحلقة حول الحمار، ذلك الحيوان الذي ارتبط...,New Media Academy Life,https://www.youtube.com/watch?v=hCM4b476p08
1,المراجعة النهائية للدوائر الكهريبة | الدحيح,حلقة انهارده عن رحلة الكهربا...\nمن قوة خفية ب...,New Media Academy Life,https://www.youtube.com/watch?v=_y_ptIWYBh4
2,تأملات في صوت جارة القمر | الدحيح,فيروز المراهقة الخجولة التي احترفت الغناء في س...,New Media Academy Life,https://www.youtube.com/watch?v=m5MSWv9nJfM
3,المراهنات | الدحيح,لماذا نراهن على لعبة نعرف أنها خاسرة؟ وما العل...,New Media Academy Life,https://www.youtube.com/watch?v=dUzWEha-plE
4,لماذا نحن هنا؟ | الدحيح,حين تختفي التعريفات الجاهزة، ويُترك الإنسان عا...,New Media Academy Life,https://www.youtube.com/watch?v=GVdMd9yJq8Q


In [6]:
def extract_id(url):
    match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
    return match.group(1) if match else None

In [7]:
youtube = build('youtube', 'v3', developerKey=api_key)
not_found_or_inaccessible = []

In [8]:
for url in data['URL']:
    video_id = extract_id(url)
    if not video_id:
        continue

    try:
        res = youtube.videos().list(part='statistics,contentDetails', id=video_id).execute()
        items = res.get('items', [])
        if not items:
            not_foun_or_inaccessible.append(url)
            continue
            
        item = items[0]

        stats = item.get('statistics', {})
        details = item.get('contentDetails', {})

        data.loc[data['URL'] == url, 'Views'] = stats.get('viewCount')
        data.loc[data['URL'] == url, 'Likes'] = stats.get('likeCount')
        data.loc[data['URL'] == url, 'Comments'] = stats.get('commentCount')
        data.loc[data['URL'] == url, 'Duration'] = details.get('duration')

    except Exception as e:
        print(f"{video_id} - error: {e}")

L0YcrQR-IEU - error: name 'not_foun_or_inaccessible' is not defined
p__sAeFp1Ms - error: name 'not_foun_or_inaccessible' is not defined
vDlG8U_Q2qE - error: name 'not_foun_or_inaccessible' is not defined
pftJmaV2nNA - error: name 'not_foun_or_inaccessible' is not defined
2hD-12AKfkU - error: name 'not_foun_or_inaccessible' is not defined
NEBG-jvABOA - error: name 'not_foun_or_inaccessible' is not defined
KZW1UyAqPSY - error: name 'not_foun_or_inaccessible' is not defined
G_9EeWbFu14 - error: name 'not_foun_or_inaccessible' is not defined
qQjVJk9h8BQ - error: name 'not_foun_or_inaccessible' is not defined
qQjVJk9h8BQ - error: name 'not_foun_or_inaccessible' is not defined


In [9]:
data['Views'] = pd.to_numeric(data['Views'], errors='coerce')
data = data.sort_values(by='Views', ascending=False, ignore_index=True)

In [10]:
data

,Title,Description,Channel,URL,Views,Likes,Comments,Duration
0,فلسطين .. حكاية الأرض | الدحيح | Palestine .. ...,هكذا بدأت الحكاية\n0:00-12:58 الطنطورة \n12:59...,New Media Academy Life,https://www.youtube.com/watch?v=f0oy-NicIgE,25832260.0,1203867,56842,PT1H2M36S
1,الدحيح - الحرب العالمية الثانية,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=dEiS_WpFuc0,23855609.0,596126,18594,PT39M14S
2,الدحيح - الفراعنة في 1602 ثانية,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=NDiAWaODyXM,12175241.0,340000,14461,PT26M43S
3,الدحيح - هتلر,"لمتابعة ""الدحيح""\nhttp://ajar.io/ryr6\n\nالدحي...",AJ+ كبريت,https://www.youtube.com/watch?v=14IzHF1zi28,12013980.0,363063,7452,PT13M57S
4,الدحيح | بناء الأهرامات,دليلك العصري لرص الحجر الأصلي\n\nتستعرض الحلقة...,Museum of The Future متحف المستقبل,https://www.youtube.com/watch?v=Gi0D5vHzimE,10421843.0,298513,6380,PT23M
...,...,...,...,...,...,...,...,...
633,Private video,This video is private.,New Media Academy Life,https://www.youtube.com/watch?v=NEBG-jvABOA,NaN,NaN,NaN,NaN
634,Private video,This video is private.,New Media Academy Life,https://www.youtube.com/watch?v=KZW1UyAqPSY,NaN,NaN,NaN,NaN
635,Private video,This video is private.,New Media Academy Life,https://www.youtube.com/watch?v=G_9EeWbFu14,NaN,NaN,NaN,NaN
636,Private video,This video is private.,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=qQjVJk9h8BQ,NaN,NaN,NaN,NaN


In [11]:
data = data[~data['Title'].str.lower().str.contains("this video is private|private video", na=False)]

In [12]:
data

,Title,Description,Channel,URL,Views,Likes,Comments,Duration
0,فلسطين .. حكاية الأرض | الدحيح | Palestine .. ...,هكذا بدأت الحكاية\n0:00-12:58 الطنطورة \n12:59...,New Media Academy Life,https://www.youtube.com/watch?v=f0oy-NicIgE,25832260.0,1203867,56842,PT1H2M36S
1,الدحيح - الحرب العالمية الثانية,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=dEiS_WpFuc0,23855609.0,596126,18594,PT39M14S
2,الدحيح - الفراعنة في 1602 ثانية,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=NDiAWaODyXM,12175241.0,340000,14461,PT26M43S
3,الدحيح - هتلر,"لمتابعة ""الدحيح""\nhttp://ajar.io/ryr6\n\nالدحي...",AJ+ كبريت,https://www.youtube.com/watch?v=14IzHF1zi28,12013980.0,363063,7452,PT13M57S
4,الدحيح | بناء الأهرامات,دليلك العصري لرص الحجر الأصلي\n\nتستعرض الحلقة...,Museum of The Future متحف المستقبل,https://www.youtube.com/watch?v=Gi0D5vHzimE,10421843.0,298513,6380,PT23M
...,...,...,...,...,...,...,...,...
623,الفقر ليس السبب في أزمة منتصف العمر | الدحيح,#الدحيح,New Media Academy Life,https://www.youtube.com/watch?v=JWQI4dQC2YQ,52476.0,2123,25,PT41S
624,ستيفن كاري هو أشهر تاجر صابون في السلة الأمريك...,#الدحيح,New Media Academy Life,https://www.youtube.com/watch?v=Qwhd6M2hueA,39332.0,1040,15,PT1M27S
625,SU Salons: Professors Don't Bite (Part1),SU Salons is a newly academic project that aim...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=h9FXU8A5Zt4,10522.0,99,5,PT4M34S
626,SU Salons: Professors Don't Bite (Part1),SU Salons is a newly academic project that aim...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=h9FXU8A5Zt4,10522.0,99,5,PT4M34S


In [13]:
data = data.copy()
data.loc[:, 'Duration_seconds'] = data['Duration'].apply(
    lambda x: isodate.parse_duration(x).total_seconds() if pd.notnull(x) else 0
)
data.loc[:, 'Duration_seconds'] = pd.to_numeric(data['Duration_seconds'], errors='coerce').astype('Int64')

In [14]:
data

,Title,Description,Channel,URL,Views,Likes,Comments,Duration,Duration_seconds
0,فلسطين .. حكاية الأرض | الدحيح | Palestine .. ...,هكذا بدأت الحكاية\n0:00-12:58 الطنطورة \n12:59...,New Media Academy Life,https://www.youtube.com/watch?v=f0oy-NicIgE,25832260.0,1203867,56842,PT1H2M36S,3756.0
1,الدحيح - الحرب العالمية الثانية,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=dEiS_WpFuc0,23855609.0,596126,18594,PT39M14S,2354.0
2,الدحيح - الفراعنة في 1602 ثانية,"لمتابعة ""الدحيح"" \nhttp://www.youtube.com/c/AJ...",AJ+ كبريت,https://www.youtube.com/watch?v=NDiAWaODyXM,12175241.0,340000,14461,PT26M43S,1603.0
3,الدحيح - هتلر,"لمتابعة ""الدحيح""\nhttp://ajar.io/ryr6\n\nالدحي...",AJ+ كبريت,https://www.youtube.com/watch?v=14IzHF1zi28,12013980.0,363063,7452,PT13M57S,837.0
4,الدحيح | بناء الأهرامات,دليلك العصري لرص الحجر الأصلي\n\nتستعرض الحلقة...,Museum of The Future متحف المستقبل,https://www.youtube.com/watch?v=Gi0D5vHzimE,10421843.0,298513,6380,PT23M,1380.0
...,...,...,...,...,...,...,...,...,...
623,الفقر ليس السبب في أزمة منتصف العمر | الدحيح,#الدحيح,New Media Academy Life,https://www.youtube.com/watch?v=JWQI4dQC2YQ,52476.0,2123,25,PT41S,41.0
624,ستيفن كاري هو أشهر تاجر صابون في السلة الأمريك...,#الدحيح,New Media Academy Life,https://www.youtube.com/watch?v=Qwhd6M2hueA,39332.0,1040,15,PT1M27S,87.0
625,SU Salons: Professors Don't Bite (Part1),SU Salons is a newly academic project that aim...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=h9FXU8A5Zt4,10522.0,99,5,PT4M34S,274.0
626,SU Salons: Professors Don't Bite (Part1),SU Salons is a newly academic project that aim...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=h9FXU8A5Zt4,10522.0,99,5,PT4M34S,274.0


In [15]:
data = data.copy()
data['Duration_seconds'] = pd.to_numeric(data['Duration_seconds'], errors='coerce')
data = data.sort_values(by='Duration_seconds', ascending=True, ignore_index=True)

In [16]:
data = data[data['Duration_seconds'] >= 200].reset_index(drop=True)

In [17]:
data = data[data['Title'] != "SU Salons: Professors Don't Bite (Part1)"].reset_index(drop=True)

In [18]:
data['Title'] = (
    data['Title']
    .str.replace('الدحيح', '', regex=False)
    .str.replace('-', '', regex=False)
    .str.replace('|', '', regex=False)
    .str.strip()
    .apply(lambda x: re.sub(r'\s+', ' ', x))
)

In [19]:
data.head(5)

,Title,Description,Channel,URL,Views,Likes,Comments,Duration,Duration_seconds
0,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313.0,22053,780,PT4M28S,268.0
1,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313.0,22053,780,PT4M28S,268.0
2,إنت جامد,"لمتابعة ""الدحيح"" \nhttp://ajar.io/ryr6j\n\nالد...",AJ+ كبريت,https://www.youtube.com/watch?v=TG3vHrlgczM,1627517.0,61436,1720,PT4M29S,269.0
3,ماذا قال عن الأب والأم والعلاقات عن بعد؟,ماذا قال الدحيح عن الأب والأم والعلاقات عن بعد...,New Media Academy Life,https://www.youtube.com/watch?v=UXbGqkCE7Qw,96514.0,2865,100,PT4M44S,284.0
4,ليه ما بنتكلمش زي الاجانب؟ (Speech Perception),لمشاهدة الحلقة الأولي من الكورة مع السلامة - ا...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=z9VlaSRmcWE,2232645.0,105597,2060,PT4M46S,286.0


In [20]:
data['Duration_seconds'] = pd.to_numeric(data['Duration_seconds'], errors='coerce')
data['Views'] = pd.to_numeric(data['Views'], errors='coerce')

data['Duration_seconds'] = data['Duration_seconds'].fillna(0).astype(int)
data['Views'] = data['Views'].fillna(0).astype(int)

In [21]:
data.head(5)

,Title,Description,Channel,URL,Views,Likes,Comments,Duration,Duration_seconds
0,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313,22053,780,PT4M28S,268
1,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313,22053,780,PT4M28S,268
2,إنت جامد,"لمتابعة ""الدحيح"" \nhttp://ajar.io/ryr6j\n\nالد...",AJ+ كبريت,https://www.youtube.com/watch?v=TG3vHrlgczM,1627517,61436,1720,PT4M29S,269
3,ماذا قال عن الأب والأم والعلاقات عن بعد؟,ماذا قال الدحيح عن الأب والأم والعلاقات عن بعد...,New Media Academy Life,https://www.youtube.com/watch?v=UXbGqkCE7Qw,96514,2865,100,PT4M44S,284
4,ليه ما بنتكلمش زي الاجانب؟ (Speech Perception),لمشاهدة الحلقة الأولي من الكورة مع السلامة - ا...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=z9VlaSRmcWE,2232645,105597,2060,PT4M46S,286


In [22]:
not_found_or_inaccessible

[]

In [23]:
data.to_excel('cleaned_d7ee7_data.xlsx', index=False)

In [24]:
data

,Title,Description,Channel,URL,Views,Likes,Comments,Duration,Duration_seconds
0,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313,22053,780,PT4M28S,268
1,ابن مجدي عبد الغني (Epigenetics),مجدي عبد الغني أفحم الناس بضربة الجزاء لكن في ...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=shkThcSCWqM,617313,22053,780,PT4M28S,268
2,إنت جامد,"لمتابعة ""الدحيح"" \nhttp://ajar.io/ryr6j\n\nالد...",AJ+ كبريت,https://www.youtube.com/watch?v=TG3vHrlgczM,1627517,61436,1720,PT4M29S,269
3,ماذا قال عن الأب والأم والعلاقات عن بعد؟,ماذا قال الدحيح عن الأب والأم والعلاقات عن بعد...,New Media Academy Life,https://www.youtube.com/watch?v=UXbGqkCE7Qw,96514,2865,100,PT4M44S,284
4,ليه ما بنتكلمش زي الاجانب؟ (Speech Perception),لمشاهدة الحلقة الأولي من الكورة مع السلامة - ا...,Da7ee7 - الدحيح,https://www.youtube.com/watch?v=z9VlaSRmcWE,2232645,105597,2060,PT4M46S,286
...,...,...,...,...,...,...,...,...,...
613,دونالد ترامب,في رحلة تحوله من رجل أعمال ونجم تليفزيون الواق...,New Media Academy Life,https://www.youtube.com/watch?v=uA-h1CmScHQ,5064246,102744,3283,PT49M42S,2982
614,ألف سنة من الإنهيار,لماذا لا يتكلم المصريون الحاليون نفس لغة المصر...,New Media Academy Life,https://www.youtube.com/watch?v=HG3xQjwenTc,3442237,84557,7442,PT51M19S,3079
615,الصعاليك,في قلب شبه الجزيرة العربية، خرج مجموعة من الرج...,New Media Academy Life,https://www.youtube.com/watch?v=WQj_OUBaMg0,4006370,95328,4472,PT53M43S,3223
616,جراح القلب,تدور الحلقة حول الجراح السير مجدي يعقوب وكيف ت...,New Media Academy Life,https://www.youtube.com/watch?v=9ZTCUyhoknU,3765475,172175,5960,PT58M39S,3519
